# 1. 데이터 로드

In [1]:
import os
import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from pytrends.request import TrendReq


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# 2. API설정

In [2]:
from api_set import APIClient

# API 설정
from utils import get_secret
BASE_URL = get_secret("BASE_URL")
CUSTOMER_ID = get_secret("CUSTOMER_ID")
API_KEY = get_secret("API_KEY")
SECRET_KEY = get_secret("SECRET_KEY")
URI = get_secret("URI")
METHOD = get_secret("METHOD")
# API 클라이언트 인스턴스 생성
api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY,URI,METHOD)


# 3. 연관검색어 수집

In [3]:
# 키 로드
from utils import load_keywords 
keywords_data = load_keywords('main_keyword.json')

from utils import get_today_date
# 오늘의 날짜 가져오기
formatted_today, day = get_today_date()


# 결과 저장 폴더 생성
from utils import make_directory

make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

In [4]:

# 검색어 리스트와 결과 저장 경로 설정
srch_keyword = ['keyword_final']  
save_path = './data/rl_srch/'  
print(api_client.base_url)

https://api.searchad.naver.com


In [5]:
srch_keyword

['keyword_final']

In [6]:
import os
import csv
import datetime
import asyncio
import pandas as pd

# 필요한 경우 비동기를 위한 nest_asyncio 적용
import nest_asyncio
nest_asyncio.apply()

from collect_keywords import collect_keywords

async def main(srch_keyword, day):
    # 오늘 날짜로 폴더 경로 생성
    folder_path = './data/rl_srch/' + datetime.datetime.now().strftime('%y%m%d')
    file_path = f"{folder_path}/collected_keywords.csv"
    
    # 폴더가 존재하는지 확인
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일이 존재하는지 확인
    if os.path.isfile(file_path):
        # 파일이 존재하면, 데이터를 읽어옵니다.
        collected_keywords_data = pd.read_csv(file_path)
    else:
        # 파일이 없으면, collect_keywords 함수를 호출해서 데이터를 수집합니다.
        collected_keywords_data = await collect_keywords(srch_keyword, day)
        # 결과를 CSV로 저장
        collected_keywords_data.to_csv(file_path, index=False)
    
    return collected_keywords_data
collected_keywords_data=asyncio.run(main(srch_keyword, day))

In [7]:
print(collected_keywords_data)

         연관키워드  월간검색수_합계 검색어
0           주식  528900.0  주식
1          미주부     390.0  주식
2        김현준대표     960.0  주식
3         퀀트투자    6710.0  주식
4         주식투자   12390.0  주식
...        ...       ...  ..
30599   수원유언공증      18.0  증여
30600   법인주소변경    1960.0  증여
30601   인터넷법무사     470.0  증여
30602    환지예정지     340.0  증여
30603  지정유언집행자      90.0  증여

[30604 rows x 3 columns]


In [8]:
sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)

# 각 '검색어'별로 분리된 DataFrame을 리스트에 저장
grouped = sorted_df.groupby('검색어')
df_list = [group for _, group in grouped]
print(len(df_list))

45


C:\Users\chohy\AppData\Local\Temp\ipykernel_22384\3376418344.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)


In [10]:
from utils import merge_and_mark_duplicates_limited
collected_keywords_data = merge_and_mark_duplicates_limited(df_list)


collected_keywords_data



,연관키워드,월간검색수_합계,검색어,중복검색어
0,달러환율,2042300.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식"
1,적금계산기,642400.0,CMA금리비교,"CMA금리비교,CMA통장"
2,다우지수,561800.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주"
3,적금,468200.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식"
4,코스닥,444900.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매..."
...,...,...,...,...
1139,S&P500지수,33500.0,테마주,"테마주,퇴직연금"
1140,TQQQ,32690.0,테마주,테마주
1141,코스피야간선물,31180.0,테마주,테마주
1142,양자컴퓨터관련주,30470.0,테마주,테마주


In [11]:
import utils

def add_client_info(collected_keywords_data, start_id_index=1):
    clients = utils.get_secret("clients")
    start_id_index = 1
    clients = utils.get_secret("clients")
    # ID와 PW 컬럼을 데이터프레임에 추가하는 로직
    total_rows = len(collected_keywords_data)
    ids = []
    pws = []

    for i in range(total_rows):
        # 현재 id 인덱스 계산 (start_id_index를 기준으로)
        current_id_index = ((i // 500) + start_id_index) % len(clients)
        current_id_key = f"id_{current_id_index}"
        
        # 현재 id와 pw 할당
        current_id = clients[current_id_key]['client_id']
        current_pw = clients[current_id_key]['client_secret']
        
        ids.append(current_id)
        pws.append(current_pw)

    # ID와 PW 컬럼 추가
    collected_keywords_data['id'] = ids
    collected_keywords_data['pw'] = pws

    return collected_keywords_data
collected_keywords_data= add_client_info(collected_keywords_data)
new_columns = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성']

for column in new_columns:
    collected_keywords_data[column] = 0
collected_keywords_data

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw
0,달러환율,2042300.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
1,적금계산기,642400.0,CMA금리비교,"CMA금리비교,CMA통장",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
2,다우지수,561800.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
3,적금,468200.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
4,코스닥,444900.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매...",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
...,...,...,...,...,...,...
1139,S&P500지수,33500.0,테마주,"테마주,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1140,TQQQ,32690.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1141,코스피야간선물,31180.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1142,양자컴퓨터관련주,30470.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8


In [12]:
def groupped_df(name,collected_keywords_data):
    grouped = collected_keywords_data.groupby(name)
    df_list = [group for _, group in grouped]
    return df_list
df_list=groupped_df('id',collected_keywords_data)
print(len(df_list))

3


In [13]:
import asyncio
import trend  

# 비동기 메인 함수 
async def trend_main(df, clients):
    # 파라미터 설정
    params = {
        "search_keywords": list(df['연관키워드']),
        "id": df['id'].iloc[0],
        "pw": df['pw'].iloc[0],
        "api_url": "https://openapi.naver.com/v1/datalab/search",
        "name": '연관검색어'
    }
    api_url = "https://openapi.naver.com/v1/datalab/search"
    
    # trend_maincode 함수 실행
    results = await trend.trend_maincode(params, clients, api_url)
    return results

async def run_all(df_list, clients):
    tasks = [trend_main(df, clients) for df in df_list]
    results = await asyncio.gather(*tasks)
    return results

clients = get_secret("clients")  
# 이벤트 루프 실행
results = asyncio.run(run_all(df_list, clients))

클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.c

In [14]:
from select_keyword import select_keyword, rising_keyword_analysis, monthly_rule

start_time = time.time()
select_periods = ['daily', 'weekly', 'month']
rising_periods=['weekly', 'month']

formatted_today, today_date = utils.get_today_date()
month_rule_list=[]
select_list=[[],[],[]]

rising_list=[[],[]]
# 월별, 주별, 일별 키워드 분석 실행

    # 각 분석 기간에 대해 결과 집합을 순회합니다.
for keyword_group in results:
    # 키워드 그룹의 각 키워드 데이터프레임에 대해 순회합니다.
    for keyword_data in keyword_group:
        # 월별 규칙을 적용하여 결과를 가져옵니다.
        monthly_data, monthly_chart, similarity_rate, rising_months = monthly_rule(keyword_data, today_date, 'month')
        if monthly_data is not None:
            # 결과 데이터프레임의 열 이름을 가져옵니다.
            column_names = monthly_data.columns
            
            # 결과 데이터프레임에서 값 리스트를 추출합니다.
            data_values_list = monthly_data[column_names].values
            # 월별 차트에 데이터 값을 추가합니다.
            monthly_chart['Indicator'] = data_values_list
            monthly_chart['InfoData'] = similarity_rate
            # 상승 월 정보를 추가합니다. 상승 월이 없는 경우 0으로 설정합니다.
            monthly_chart['RisingMonth'] = rising_months[0] if len(rising_months) > 0 else 0
            
            # 최종 결과 리스트에 수정된 월별 차트를 추가합니다.
            month_rule_list.append(monthly_chart)
                
# 주별, 월별 상승 키워드 분석 실행
rising_analysis_periods = ['weekly', 'month']
i=0
for period in rising_analysis_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            rising_tmp, rising_graph, rising_info = rising_keyword_analysis(keyword_df, today_date, period)
            if rising_tmp is not None:
                column_names=rising_tmp.columns
                data_values_list = rising_tmp[column_names].values
                rising_graph['Indicator'] = data_values_list
                rising_graph['InfoData'] = rising_info

                rising_list[i].append(rising_graph)
    i=i+1

    


i = 0
# 일별, 주별, 월별 키워드 선택 실행
for period in select_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            selected_tmp, selected_graph, selected_info = select_keyword(keyword_df, today_date, period)
            if selected_graph is not None:
                # 데이터프레임의 열 이름을 출력합니다.
                selected_graph['InfoData'] = selected_info
                select_list[i].append(selected_graph)
            else:
                pass
    i += 1

end_time = time.time()
print(f"Analysis completed in {end_time - start_time} seconds.")

월별 규칙성 키워드 발견 : 소득공제
월별 규칙성 키워드 발견 : 연말정산소득공제
월별 규칙성 키워드 발견 : 체크카드혜택
월별 규칙성 키워드 발견 : 연말정산
월별 규칙성 키워드 발견 : 샘플
월별 규칙성 키워드 발견 : 연말정산하는법
월별 규칙성 키워드 발견 : AIG손해보험
월별 규칙성 키워드 발견 : AIG
월별 규칙성 키워드 발견 : 알바
월별 규칙성 키워드 발견 : 일자리
월별 규칙성 키워드 발견 : 앱
월별 규칙성 키워드 발견 : 중고차어플
월별 규칙성 키워드 발견 : 근로계약서
월별 규칙성 키워드 발견 : 소득공제용연금저축
월별 규칙성 키워드 발견 : 직계존속
월별 규칙성 키워드 발견 : 신용카드발급
월별 규칙성 키워드 발견 : 거실벽인테리어
월별 규칙성 키워드 발견 : 동물그림
월별 규칙성 키워드 발견 : 석상
월별 규칙성 키워드 발견 : 명함제작
월별 규칙성 키워드 발견 : 주택관리사
월별 규칙성 키워드 발견 : 월세
월별 규칙성 키워드 발견 : 부동산매매
월별 규칙성 키워드 발견 : 상속세세율
월별 규칙성 키워드 발견 : 증여세율
월별 규칙성 키워드 발견 : 한정승인
월별 규칙성 키워드 발견 : 상속포기신고서
월별 규칙성 키워드 발견 : 상속포기신청
월별 규칙성 키워드 발견 : 연말정산간소화
월별 규칙성 키워드 발견 : 원천징수
월별 규칙성 키워드 발견 : 부가세
월별 규칙성 키워드 발견 : 법무사
월별 규칙성 키워드 발견 : 차용증양식
월별 규칙성 키워드 발견 : 전세권설정
월별 규칙성 키워드 발견 : 등기
월별 규칙성 키워드 발견 : 마석역
월별 규칙성 키워드 발견 : 아파트월세
월별 규칙성 키워드 발견 : 평생교육원
월별 규칙성 키워드 발견 : 해외선물
월별 규칙성 키워드 발견 : 자동차보험
월별 규칙성 키워드 발견 : KB자동차보험
월별 규칙성 키워드 발견 : 케이비손해보험
주별 지속상승 키워드 발견 : SOXL주가
주별 지속상승 키워드 발견 : 국내채권ETF
주별 지속상승 키워드 발견 : 인도증시
주별 지속상승 키워드 발견 : 간병

In [19]:
# 월별...이런거에 1 배정
def update_keywords_flag(dataframe, data_list, flag_name):
    unique_associated_keywords = set()
    for df in data_list:
        unique_associated_keywords.update(df['연관검색어'].unique())
    for index, row in dataframe.iterrows():
        if row['연관키워드'] in unique_associated_keywords:
            dataframe.at[index, flag_name] = 1


flags_and_lists = [
    ("일별급상승", select_list[0]),
    ("주별급상승", select_list[1]),
    ("월별급상승", select_list[2]),
    ("주별지속상승", rising_list[0]),
    ("월별지속상승", rising_list[1]),
    ("월별규칙성", month_rule_list),
]

for flag_name, data_list in flags_and_lists:
    update_keywords_flag(collected_keywords_data, data_list, flag_name)


In [87]:
def select_and_process(df, condition, type_label):
    # 조건에 맞는 데이터 선택
    selected_df = df[df[condition] == 1].copy()
    selected_df['유형'] = type_label
    
    # 불필요한 열 삭제
    drop_columns = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성', 'id', 'pw', '검색어', '월간검색수_합계', '중복검색어']
    selected_df.drop(drop_columns, axis=1, inplace=True)
    
    # 인덱스 초기화
    selected_df.reset_index(drop=True, inplace=True)
    
    return selected_df

# 각 조건에 대해 함수 호출
conditions_and_labels = [
    ('일별급상승', '일별급상승'),
    ('주별급상승', '주별급상승'),
    ('월별급상승', '월별급상승'),
    ('주별지속상승', '주별지속상승'),
    ('월별지속상승', '월별지속상승'),
    ('월별규칙성', '월별규칙성'),
]

# 빈 리스트를 생성하여 각 조건에 대한 결과를 저장
selected_dfs = []

for condition, label in conditions_and_labels:
    selected_df = select_and_process(collected_keywords_data, condition, label)
    selected_dfs.append(selected_df)
concatenated_df = pd.concat(selected_dfs, ignore_index=True)

In [107]:
# 결과를 저장할 빈 리스트 초기화
matching_indices = []

# results 리스트를 반복하며 각 데이터프레임 탐색
for i, inner_list in enumerate(results):
    for j, df in enumerate(inner_list):
        asset_name = df.columns[0]  # 각 데이터프레임의 열 이름 (가상자산 이름)
        
        # concatenated_df에서 asset_name과 일치하는 연관키워드를 찾고, 해당 유형을 가져옴
        matching_rows = concatenated_df[concatenated_df['연관키워드'] == asset_name]
        
        # 일치하는 연관키워드가 있다면
        if not matching_rows.empty:
            for _, row in matching_rows.iterrows():
                type_name = row['유형']  # 일치하는 유형 이름
                # 결과 리스트에 인덱스와 유형을 추가
                matching_indices.append([i, j, type_name])


[0, 7, '월별급상승']

In [149]:
results[0][0]

,달러환율
date,
2020-03-07,10.50296
2020-03-08,8.70288
2020-03-09,32.45485
2020-03-10,32.28908
2020-03-11,26.24714
...,...
2024-03-02,12.69856
2024-03-03,11.73118
2024-03-04,25.15213


# graph_result csv 

In [150]:
graph_result = pd.DataFrame()

for idx in matching_indices:
    temp_df = results[idx[0]][idx[1]].reset_index()
    name = temp_df.columns[1]
    temp_df['연관검색어'] = name
    temp_df.rename(columns={'date': '검색일자', name: '검색량'}, inplace=True)
    temp_df['유형'] = idx[2]
    temp_df['오늘일자'] = datetime.today().strftime('%Y-%m-%d')
    
    # '검색일자'를 datetime 객체로 변환
    temp_df['검색일자'] = pd.to_datetime(temp_df['검색일자'])
    
    # 유형에 따라 데이터 필터링 (뒤에서부터 필터링)
    if '주별' in temp_df['유형'].iloc[0]:
        # 주별 데이터: 뒤에서부터 7일 간격으로 필터링
        filter_index = range(len(temp_df)-1, -1, -7)
    elif '월별' in temp_df['유형'].iloc[0]:
        # 월별 데이터: 뒤에서부터 28일 간격으로 필터링
        filter_index = range(len(temp_df)-1, -1, -28)
    else:
        filter_index = range(len(temp_df))
    
    # 필터링된 인덱스에 해당하는 행만 선택
    temp_df = temp_df.iloc[filter_index]
    
    # 결과 DataFrame에 추가
    graph_result = pd.concat([graph_result, temp_df], ignore_index=True)

print(graph_result.head(500))


          검색일자       검색량     연관검색어     유형        오늘일자
0   2024-03-06  50.60636      파킹통장  월별급상승  2024-03-08
1   2024-02-07  44.47369      파킹통장  월별급상승  2024-03-08
2   2024-01-10  55.16378      파킹통장  월별급상승  2024-03-08
3   2023-12-13  34.85822      파킹통장  월별급상승  2024-03-08
4   2023-11-15   8.74368      파킹통장  월별급상승  2024-03-08
..         ...       ...       ...    ...         ...
495 2020-05-04   0.93896  레고켐바이오주가  일별급상승  2024-03-08
496 2020-05-05   0.13101  레고켐바이오주가  일별급상승  2024-03-08
497 2020-05-06   1.38115  레고켐바이오주가  일별급상승  2024-03-08
498 2020-05-07   1.65411  레고켐바이오주가  일별급상승  2024-03-08
499 2020-05-08   1.05360  레고켐바이오주가  일별급상승  2024-03-08

[500 rows x 5 columns]


In [141]:
matching_indices

[[0, 7, '월별급상승'],
 [0, 9, '월별급상승'],
 [0, 15, '월별급상승'],
 [0, 20, '월별급상승'],
 [0, 33, '주별급상승'],
 [0, 41, '월별급상승'],
 [0, 45, '일별급상승'],
 [0, 45, '주별급상승'],
 [0, 45, '월별급상승'],
 [0, 46, '주별급상승'],
 [0, 54, '월별급상승'],
 [0, 54, '월별지속상승'],
 [0, 61, '월별급상승'],
 [0, 61, '월별지속상승'],
 [0, 63, '월별급상승'],
 [0, 63, '월별지속상승'],
 [0, 64, '월별급상승'],
 [0, 64, '월별지속상승'],
 [0, 75, '월별급상승'],
 [0, 75, '월별지속상승'],
 [0, 77, '월별규칙성'],
 [0, 78, '월별급상승'],
 [0, 78, '월별지속상승'],
 [0, 82, '월별규칙성'],
 [0, 101, '월별규칙성'],
 [0, 102, '월별급상승'],
 [0, 109, '월별급상승'],
 [0, 119, '월별규칙성'],
 [0, 127, '월별급상승'],
 [0, 127, '주별지속상승'],
 [0, 128, '주별급상승'],
 [0, 128, '월별급상승'],
 [0, 131, '월별급상승'],
 [0, 134, '월별급상승'],
 [0, 137, '월별규칙성'],
 [0, 139, '월별지속상승'],
 [0, 143, '월별급상승'],
 [0, 143, '월별지속상승'],
 [0, 157, '월별지속상승'],
 [0, 166, '주별지속상승'],
 [0, 171, '월별급상승'],
 [0, 194, '월별급상승'],
 [0, 202, '주별급상승'],
 [0, 202, '월별급상승'],
 [0, 202, '주별지속상승'],
 [0, 204, '월별급상승'],
 [0, 204, '월별지속상승'],
 [0, 205, '월별규칙성'],
 [0, 212, '주별급상승'],
 [0, 212, '월별급상승'],
 [0, 212, '월별

In [54]:
unique_associated_keywords = set()
for df in select_list[0]:
    unique_associated_keywords.update(df['연관검색어'].unique())
for index, row in collected_keywords_data.iterrows():
    if row['연관키워드'] in unique_associated_keywords:
        collected_keywords_data.at[index, '일별급상승'] = 1
unique_associated_keywords = set()
for df in select_list[1]:
    print(df)
    unique_associated_keywords.update(df['연관검색어'].unique())
for index, row in collected_keywords_data.iterrows():
    if row['연관키워드'] in unique_associated_keywords:
        collected_keywords_data.at[index, '주별급상승'] = 1

unique_associated_keywords = set()
for df in select_list[2]:
    unique_associated_keywords.update(df['연관검색어'].unique())
for index, row in collected_keywords_data.iterrows():
    if row['연관키워드'] in unique_associated_keywords:
        collected_keywords_data.at[index, '월별급상승'] = 1
unique_associated_keywords = set()
for df in rising_list[0]:

    unique_associated_keywords.update(df['연관검색어'].unique())
print(unique_associated_keywords)
for index, row in collected_keywords_data.iterrows():
    if row['연관키워드'] in unique_associated_keywords:
        collected_keywords_data.at[index, '주별지속상승'] = 1

unique_associated_keywords = set()
for df in rising_list[1]:
    unique_associated_keywords.update(df['연관검색어'].unique())
for index, row in collected_keywords_data.iterrows():
    if row['연관키워드'] in unique_associated_keywords:
        collected_keywords_data.at[index, '월별지속상승'] = 1
unique_associated_keywords = set()

for df in month_rule_list:
    unique_associated_keywords.update(df['연관검색어'].unique())
print(unique_associated_keywords)
for index, row in collected_keywords_data.iterrows():
    if row['연관키워드'] in unique_associated_keywords:
        collected_keywords_data.at[index, '월별규칙성'] = 1


{'한돈시세', '건강종합보험', '헤데라코인', '입원보험', '카드이벤트', '콜드월렛', '와이즈버즈', '자동차종합보험', '가상자산', '부산금소매가', '금1돈', '금한돈', '솔라나시세', '성취', '3.3%계산기', '비트코인ETF', '테더코인', '오늘상장주', '금투자', 'SOXL주가', 'THE건강보험', '금1돈시세', '솔루션', '세이코인', '캐피탈', '국내채권ETF', 'ISA계좌개설', '암호화폐거래소순위', '간병인보험', '오늘상장주식', '퇴직금세금', '인도증시', '자동차보험', 'ETF수익률', '베어유'}
